In [9]:
import pandas as pd
import numpy as np

In [10]:
def map_age(age):
    if age < 60: return '< 60'
    elif age >= 60 and age < 70: return '60-69'
    elif age >= 70 and age < 80: return '70-69'
    elif age >= 80: return '> 80'
    

def map_age2(age):
    if age < 60: return '< 60'
    elif age >= 60: return '> 60'
    
def map_sexo(sexo):
    
    if sexo == 'Masc': return 'M'
    elif sexo == 'Fem': return 'F'
    elif sexo == 'Mas': return 'M'
    elif sexo == 'FEM': return 'F'
    else: return 'Não informado'

In [11]:
data_obitos = pd.read_csv("data_obitos/Obitos_Sergipe.csv")

In [12]:
data_obitos.rename(columns={
    'IDADE': 'idade', 
    'SEXO': 'sexo', 
    'MUNICÍPIO': 'municipio', 
    'COMORBIDADE': 'comorbidade', 
    'DATA ÓBITO': 'data_obito',
    'DATA CONFIRMAÇÃO': 'data_confirmacao',
},
                  inplace=True)

data_obitos['idade_faixa'] = data_obitos['idade'].apply(map_age)        
data_obitos['sexo'] = data_obitos['sexo'].apply(map_sexo)
data_obitos

,idade,sexo,municipio,comorbidade,data_obito,data_confirmacao,idade_faixa
0,42,F,São Cristóvão,Sem Comorbidade,27/06/2021,17/06/2021,< 60
1,60,F,Tobias Barreto,Sem Comorbidade,27/06/2021,07/06/2021,60-69
2,82,F,Aracaju,"HIPERTENSAO ARTERIAL SISTEMICA, DIABETES MELLI...",26/06/2021,16/06/2021,> 80
3,81,F,Aracaju,Sem Comorbidade,26/06/2021,11/06/2021,> 80
4,84,M,Barra dos Coqueiros,Acidente Vascular Encefálico,26/06/2021,15/06/2021,> 80
...,...,...,...,...,...,...,...
5647,66,M,Aracaju,"Diabetes mellitus, Hipertensão, Obesidade",06/04/2020,01/04/2020,60-69
5648,90,F,Aracaju,Hipertensão arterial,04/04/2020,04/04/2020,> 80
5649,61,F,Aracaju,"Diabetes mellitus, Hipertensão arterial, cardi...",02/04/2020,30/03/2020,60-69
5650,60,M,Aracaju,"Hipertensão Arterial, Tabagismo",02/04/2020,01/04/2020,60-69


In [13]:
data_process = data_obitos.drop(columns=['sexo','municipio','comorbidade','data_confirmacao'])
data_process['data_obito'] = pd.to_datetime(data_process.data_obito, format='%d/%m/%Y')
data_process['data_obito_m_y'] = data_process['data_obito'].dt.strftime('%Y-%m')
data_count = data_process[['data_obito_m_y','idade']].groupby(by=['data_obito_m_y']).count()
data_process = data_process[['data_obito_m_y','idade_faixa','idade']].groupby(by=['data_obito_m_y','idade_faixa']).count()
for (index, rows) in data_process.iterrows():
    data_process.at[index, 'percent'] = float(rows['idade'] / data_count.loc[index[0]].idade)
data_process.reset_index(inplace=True)
data_process.rename(columns={'idade': 'total'}, inplace=True)
data_process

,data_obito_m_y,idade_faixa,total,percent
0,2020-04,60-69,4,0.307692
1,2020-04,70-69,2,0.153846
2,2020-04,< 60,3,0.230769
3,2020-04,> 80,4,0.307692
4,2020-05,60-69,49,0.234450
5,2020-05,70-69,41,0.196172
6,2020-05,< 60,78,0.373206
7,2020-05,> 80,41,0.196172
8,2020-06,60-69,124,0.203947
9,2020-06,70-69,142,0.233553


In [14]:
#data_final = data_process.drop(columns=['total'])
data_final = data_process.pivot_table(values=['percent', 'total'], index=['data_obito_m_y'], columns='idade_faixa')
data_final.reset_index(inplace=True)
data_final.rename(columns={'data_obito_m_y': 'data', '60-69': 'faixa_6069', '70-69': 'faixa_7079', '< 60': 'faixa_menor_60', '> 80': 'faixa_maior_80', '> 60': 'faixa_maior_60'}, inplace=True)
data_final.columns = data_final.columns.get_level_values(0) + "_" + data_final.columns.get_level_values(1)
data_final.rename(columns={'data_': 'data'},inplace=True)
data_final

,data,percent_faixa_6069,percent_faixa_7079,percent_faixa_menor_60,percent_faixa_maior_80,total_faixa_6069,total_faixa_7079,total_faixa_menor_60,total_faixa_maior_80
0,2020-04,0.307692,0.153846,0.230769,0.307692,4,2,3,4
1,2020-05,0.234450,0.196172,0.373206,0.196172,49,41,78,41
2,2020-06,0.203947,0.233553,0.325658,0.236842,124,142,198,144
3,2020-07,0.221485,0.265252,0.278515,0.234748,167,200,210,177
4,2020-08,0.226300,0.247706,0.247706,0.278287,74,81,81,91
5,2020-09,0.190184,0.233129,0.306748,0.269939,31,38,50,44
6,2020-10,0.251572,0.245283,0.226415,0.276730,40,39,36,44
7,2020-11,0.266055,0.275229,0.201835,0.256881,29,30,22,28
8,2020-12,0.176000,0.244000,0.284000,0.296000,44,61,71,74
9,2021-01,0.227848,0.223629,0.345992,0.202532,54,53,82,48


In [15]:
data_final['increase_6069'] = data_final['total_faixa_6069'].pct_change()
data_final['increase_7079'] = data_final['total_faixa_7079'].pct_change()
data_final['increase_menor_60'] = data_final['total_faixa_menor_60'].pct_change()
data_final['increase_maior_80'] = data_final['total_faixa_maior_80'].pct_change()
data_final.fillna(0, inplace=True)
data_final

,data,percent_faixa_6069,percent_faixa_7079,percent_faixa_menor_60,percent_faixa_maior_80,total_faixa_6069,total_faixa_7079,total_faixa_menor_60,total_faixa_maior_80,increase_6069,increase_7079,increase_menor_60,increase_maior_80
0,2020-04,0.307692,0.153846,0.230769,0.307692,4,2,3,4,0.000000,0.000000,0.000000,0.000000
1,2020-05,0.234450,0.196172,0.373206,0.196172,49,41,78,41,11.250000,19.500000,25.000000,9.250000
2,2020-06,0.203947,0.233553,0.325658,0.236842,124,142,198,144,1.530612,2.463415,1.538462,2.512195
3,2020-07,0.221485,0.265252,0.278515,0.234748,167,200,210,177,0.346774,0.408451,0.060606,0.229167
4,2020-08,0.226300,0.247706,0.247706,0.278287,74,81,81,91,-0.556886,-0.595000,-0.614286,-0.485876
5,2020-09,0.190184,0.233129,0.306748,0.269939,31,38,50,44,-0.581081,-0.530864,-0.382716,-0.516484
6,2020-10,0.251572,0.245283,0.226415,0.276730,40,39,36,44,0.290323,0.026316,-0.280000,0.000000
7,2020-11,0.266055,0.275229,0.201835,0.256881,29,30,22,28,-0.275000,-0.230769,-0.388889,-0.363636
8,2020-12,0.176000,0.244000,0.284000,0.296000,44,61,71,74,0.517241,1.033333,2.227273,1.642857
9,2021-01,0.227848,0.223629,0.345992,0.202532,54,53,82,48,0.227273,-0.131148,0.154930,-0.351351


In [16]:
data_final.to_csv("output/obitos_idade_mes.csv", index=False)